In [47]:
import math
import numpy as np
import sympy as sp
import cn_utils as cn
import plotly.graph_objects as go

In [2]:
x, y, z, t = sp.symbols('x y z t')

# Definição de erro

Qualquer diferença entre o valor exato (real) de uma quantidade com o valor aproximado é considerado um erro
$$
\text{Erro = Valor verdadeiro - Valor aproximado}
$$
O problema desta definição é que não considera a magnitude das quantidades. Errar 1cm em 1m é muito grande em compração de errar 1cm em 1km. Para resolver esse problema temos o *erro relativo fracionario verdadeiro*

$$
\text{Erro relativo fracionário verdadeiro} = \frac{\text{valor verdadeiro - valor aproximado}}{\text{valor verdadeiro}} = \frac{\text{erro verdadeiro}}{\text{valor verdadeiro}}
$$
E muitas vezes falamos de uma porcentagem, *erro relativo porcentual verdadeiro*
$$
\varepsilon_t = \frac{\text{erro verdadeiro}}{\text{valor verdadeiro}} \times 100\%
$$
Mas temos um problema GRANDE! No mundo real, geralmente, a gente não sabe o valor verdadeiro de uma quantidade. Alias, se soubéssemos do valor verdadeiro não precisavamos dessa busca numerica. Nessas situações vamos normalizar o erro usando a melhor estimativa disponivel do valor verdadeiro,
$$
\varepsilon_a = \frac{\text{erro aproximado}}{\text{aproximação}} \times 100\%
$$
onde o subscrito $a$ significa que o erro foi normalizado por um valor aproximado. Em muitos casos, usamos um metodo iterativo para calcular uma quantidade, melhorando a aproximação a cada interação. Nestes casos, o erro relativo porcentual é determinado de acordo com
$$
\varepsilon_a = \frac{\text{aproximado atual - aproximação prévia}}{\text{aproximação atual}} \times 100\%
$$
Na prática, tem uma tolerancia para erro de um calculo, $\varepsilon_s$. Ou seja, o criterio para parada de um processo iterativo é que o erro seja menor dessa tolerancia,
$$
|\varepsilon_a| < \varepsilon_s
$$
Também é conveniente relacionar esses erros ao número de algarismos significativos na aproximação. Pode ser mostrado que, se o seguinte critério for satisfeito, é possível ter certeza de que o resultado é correto até pelo menos $n$ algarismos significativos
$$
\varepsilon_s = (0.5 \times 10^{2-n}) \%
$$



## **Exemplo:  Estimativas de Erros para Métodos Iterativos**


**Enunciado do Problema**. Em matemática, as funções, em geral, podem ser representadas por séries infinitas. Por exemplo, a função exponencial pode ser calculada usando-se
$$
e^x = 1 + x + \frac{x^2}{2} + \frac{x^3}{3!} + \cdot\cdot\cdot + \frac{x^n}{n!}
$$
Portanto, conforme mais termos forem adicionados em sequência, a aproximação se torna
uma estimativa cada vez melhor do valor verdadeiro de $e^x$ . A Equação acima é chamada
de *expansão em série de Maclaurin*.
Começando com a versão mais simples,  $e^x = 1$, some um termo de cada vez para estimar $e^{0,5}$. Depois que cada termo for adicionado, calcule o erro verdadeiro e o erro relativo porcentual aproximado. Observe que o valor verdadeiro é $e^{0.5}= 1.648721 . . . .$ Adicione termos até que o valor absoluto do erro estimado aproximado $\varepsilon_a$ esteja dentro do critério de erro pré-especificado $\varepsilon_s$ que garanta três algarismos significativos.

**Resolução:**
Para ter uma precisão até pelo menos tres algarismos significativos temos que colocar uma tolerancia de

$$ \varepsilon_s = (0.5 \times 10^{2-3})\% = 0.05 \% $$

A primeira estimativa é manter só o primeiro termo da expansão que é 1. para calcular o resto podemos usar a função `taylor_expansion` no modulo `cn_utils`.

In [3]:
f = sp.exp(x)

e_t_2 = (f.subs(x, 0.5) - cn.taylor_expansion(f, 0, 1).subs(x, 0.5))/f.subs(x, 0.5)
e_t_3 = (f.subs(x, 0.5) - cn.taylor_expansion(f, 0, 2).subs(x, 0.5))/f.subs(x, 0.5)
e_t_4 = (f.subs(x, 0.5) - cn.taylor_expansion(f, 0, 3).subs(x, 0.5))/f.subs(x, 0.5)


print(f"O erro da segunda estimativa é: {e_t_2}")
print(f"O erro da terceira estimativa é: {e_t_3}")
print(f"O erro da quarta estimativa é: {e_t_4}")

a segunda estimativa é: 0.0902040104310499
a terceira estimativa é: 0.0143876779669707
a quarta estimativa é: 0.00175162255629090


In [4]:
f = sp.exp(x)
e_s = 0.05/100
valor_anterior = cn.taylor_expansion(f, 0, 0).subs(x, 0.5)
i = 1
e_t = 10000
while e_t > e_s:
    valor_atual = cn.taylor_expansion(f, 0, i).subs(x, 0.5)
    e_t = (valor_atual - valor_anterior)/valor_atual
    print(e_t)
    i += 1
    valor_anterior = valor_atual


print(f"Para ter um erro menor do que {e_s} precisamos de incluir {i} da expansion e o erro relativo fracionario verdadeiro sera {e_t}")

0.333333333333333
0.0769230769230769
0.0126582278481012
0.00157977883096371
0.000157952930026843
Para ter um erro menor do que 0.0005 precisamos de incluir 6 da expansion e o erro relativo fracionario verdadeiro sera 0.000157952930026843


# A origem dos erros

Os erros numericos que aparecem em uma modelagem tem duas origens:

- Arredondamento
- Truncamento 

O **arredondamento** tem origem na arquitetura binaria dos computadores. Ou seja, temos uma limitação computacional para representar um numero extremamente pequena ou extremamente grande no computador. Não tem como definir um numero menor do que um certo limite no computador e perto deste limite as contas ficam muito instaveis. 

O **truncamente** tem relação dos a aproximação que usamos na hora de modelar um fenomeno. Por exemplo, trabalhar com os primeiros termos da expansão de Taylor (vamos conhecer mais pela frente) em vez de trabalhar com um serie infinito.


Prestem atenção que esses erros são inevitaveis e não tem relação com os possiveis erros logicos na hora de modelar um fenomeno. 

# Erros de arrendodamento
Os erros de arredondamento se originam do fato de que o computador mantém apenas um número fixo de algarismos significativos durante os cálculos. Números como $\pi$, e ou $\sqrt 7$ não podem ser expressos por um número fixo de algarismos significativos. Portanto, eles não podem ser representados exatamente por um computador. Além disso, como os computadores usam uma representação na base 2, não podem representar precisamente certos números exatos na base 10. A discrepância introduzida por essa omissão de algarismos significativos é chamada de erro de
arredondamento.


<img src="./images/number_represent.jpg" style="width:400px;height:400px;">

### Representação de numeros inteiros
Para encontrar a representação binaria de um numbero **inteiro** em representação decimal tem o seguinte procedimento

<img src="./images/decimal2binary.svg" style="width:400px;height:400px;">

In [5]:
cn.dec2bin(47)

'101111'

O resultado acima mostra que um computador precisa de 6 bits para armazenar o numero 47. O procedimento para os numeros racionais é diferente que vamos ver mais na frente. 

### **Exemplo: Intervalo de Inteiros**

**Enunciado do Problema.** Determine o intervalo dos inteiros na base 10 que pode ser
representado em um computador de 16 bits.

**Resolução:**
Dos 16 bits, o primeiro bit representa o sinal. Os restantes 15 bits podem conter números binários de 0 a 111111111111111. O limite superior pode ser convertido para um inteiro decimal, como

$$
(1 \times 2^{14}) + (1 \times 2^{13}) + \cdot\cdot\cdot + (1 \times 2^{1}) + (1 \times 2^{0})
$$
que é igual a 32.767 (observe que essa expressão pode ser calculada simplesmente como
$2^{15} - 1$). Portanto, uma palavra no computador de 16 bits pode armazenar inteiros decimais variando de -32.767 a 32.767. Além disso, como o zero já foi definido como
0000000000000000, é redundante usar o número 1000000000000000 para definir um
“menos zero”. Portanto, ele é normalmente usado para representar um número negativo
adicional: -32.768, e o intervalo é de -32.768 a 32.767.

### **Representação em ponto flutuante**
As quantidades fracionárias são representadas
tipicamente em computadores usando-se a forma de ponto flutuante. Nessa abordagem, o
número é expresso como uma parte fracionária, chamada mantissa ou significando, e uma
parte inteira, chamada de expoente ou característica, como em
$$
m\cdot b^e \sim (b, e, m)
$$
onde m é a *mantissa*, b é a base do sistema numérico que está sendo usado, e $e$ é o *expoente*. Por exemplo, o número 156,78 poderia ser representado como $1,5678 \times 10^3$ em um sistema de ponto flutuante na base 10. Essa forma de representar se chama representação cientifica. Aqui vamos usar a seguinte normalização para $m$ (diferente do livro de Chapra) 
$$
1 \le m < b
$$
onde $b$ é a base. Por exemplo, para um sistema na base 10, $m$ iria variar entre \[1 , 10\), e
para um sistema na base 2, entre \[1 , 2\).

<img src="./images/floating.jpg" style="width:600px;height:200px;">


O Python usa a **precisão dupla** que é um padrão definido pelo IEEE para respresentar os numeros racionais. Neste padrão se usa 64 bits para representar um numero real/racional. 

<img src="./images/ieee_double.png" style="width:600px;height:200px;">

Neste padrão, o *expoente* pode ter qualquer valor no conjunto de numeros naturais no intervalo 
$$
\mathbb{Z} \sim \{-1022, \cdot\cdot\cdot, -2, -1, 0, 1, 2, \cdot\cdot\cdot, 1023 \}
$$
Então, vamos ter numeros na escala de $2^{-1022}$ e $2^{1023}$ que são realmente grandes. Agora a questão é como representar a mantissa como bits já que existem infinitos numeros no intervalo \[1,2\). O truque é dividir esse intevalo em espaços pequenos. É aqui que surge o conceito de precisão de maquina (ou epsilon da maquina). Ou seja, vamos ter o seguinte conjunto de numeros
$$
\{1, 1+\epsilon, 1+2\epsilon, \cdot\cdot\cdot 1+(N-1)\epsilon\}
$$
onde $1 + N \epsilon\ = 2$. No padrão de precisão dupla o $N=2^{52} \sim 2.2\times 10^{-16}$. Neste caso, o epsilon da maquina é $\epsilon = 2^{-52}$.
Para representar os numeros negativos vamos dedicar o primeiro bit. 
$$
\pm m\cdot b^e \sim (\pm, b, e, m)
$$
Agora vamos ver como funciona nessa representação. 


| expoente(11bits) | mantissa(52bits)     | resultado(decimal)    |   
|----------|--------------|--------------|
| -1        | 1            | $\frac{1}{2}$          |  
| -1        | 1+$\epsilon$   | $\frac{1}{2} + \frac{\epsilon}{2}$   | 
| -1        | 1+$2\epsilon$  | $\frac{1}{2} + \frac{\epsilon}{4}$   |  
|  ....  |           ....    |        ....       |   
| -1        | 2 - $\epsilon$ | 1 - $\frac{\epsilon}{2}$ |   
| 0        | 1            | 1            |  
| 0        | 1+$\epsilon$   | 1+$\epsilon$   | 
| 0        | 1+$2\epsilon$  | 1+$2\epsilon$  |  
|  ....  |           ....    |        ....       |   
| 0        | 2 - $\epsilon$ | 2 - $\epsilon$ |   
| 1        | 1            | 2            |  
| 1        | 1+$\epsilon$ | 2+$2\epsilon$ |   
| 1        | 1+$2\epsilon$ | 2 + $4\epsilon$ |   
|  ....  |           ....    |        ....       | 
| 1        | 2 - $\epsilon$ | 4 - $2\epsilon$ |   
| 2        | 1            | 4            |  
| 2        | 1+$\epsilon$ | 4+$4\epsilon$ |   
| 2        | 1+$2\epsilon$ | 4 + $8\epsilon$ |   
|  ....  |           ....    |        ....       | 
| 2        | 2 - $\epsilon$ | 8 - $4\epsilon$ |   





In [6]:
print(np.finfo(float).eps)
# 2.22044604925e-16

print(np.finfo(np.float32).eps)
# 1.19209e-07

print(np.finfo(np.float64).eps)
# 1.19209e-07


print(np.finfo(np.float128).epsneg)

print(np.finfo(np.float64).max)
print(np.finfo(np.float64).maxexp)
print(np.finfo(np.float64).nmant)
print(np.finfo(np.float64).nexp)
print(np.finfo(np.float64).precision)

2.220446049250313e-16
1.1920929e-07
2.220446049250313e-16
5.42101086242752217e-20
1.7976931348623157e+308
1024
52
11
15


In [7]:
arr16 = np.array([1.], dtype=np.float16)
arr128 = np.array([1.], dtype=np.float128)
num = np.array([1e-5], dtype=np.float128)

for i in range(1000):
    arr16[0] = arr16[0]* 0.01
    arr128[0] = arr128[0] * 0.01

print(arr16[0])
print(arr128[0])

0.0
1.0000000000000208164e-2000


In [8]:
arr16[0] == 0

True

In [9]:
arr128[0] == 0

False

In [10]:
arr16 = np.array([1.], dtype=np.float16)
arr128 = np.array([1.], dtype=np.float128)
num = np.array([1e-5], dtype=np.float128)

for i in range(1000):
    arr16[0] = arr16[0] + num[0]
    arr128[0] = arr128[0] + num[0]

print(arr16[0])
print(arr128[0])

1.0
1.0100000000000000262


In [11]:
arr = np.array([1.0])
print(arr.dtype)

float64


## Cancelamento na subtração
Um exemplo deste problema aparece na hora de calcular as raizes de uma equação quadratica usando a famosa formula
$$
x_1, x_2 = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
$$
Quando os valores de $a, c$ e $b^2$ são de escalas muuito diferentes

### **Exemplo:** Cancelamento na Subtração
Calcule os valores das raízes de uma equação quadrática
com $a =1$ , $b = 3000.001$ e $c = 3$. Compare os valores calculados com as raízes verdadeiras $x_1 = -0,001$ e $x_2 =-3000$.


In [24]:
#usando half precision (32bits)
a16 = np.array([1.], dtype=np.float16)
b16 = np.array([3000.001], dtype=np.float16)
c16 = np.array([3.], dtype=np.float16)
x1_16 = np.array([1.], dtype=np.float16)
x2_16 = np.array([1.], dtype=np.float16)

x1_16[0] = (-b16[0] + np.sqrt(b16[0]**2 - 4*a16[0]*c16[0]))/(2*a16[0])
x2_16 = (-b16 - np.sqrt(b16**2 - 4*a16*c16))/(2*a16)

print(x1_16)
print(x2_16) 

[-0.001]
[-inf]


<ipython-input-24-cd10744934dc>:9: RuntimeWarning: overflow encountered in square
  x2_16 = (-b16 - np.sqrt(b16**2 - 4*a16*c16))/(2*a16)


In [25]:
(-b16[0] - np.sqrt(b16[0]**2 - 4*a16[0]*c16[0]))/(2*a16[0])

-2999.998999999667

In [21]:
#usando double precision (64bits)
a = 1
b = 3000.001
c = 3
x1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
x2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

print(x1)
print(x2)

-0.0009999999999763531
-3000.0


In [22]:
format(b, ".53f")

'3000.00100000000020372681319713592529296875000000000000000'

In [28]:
#usando extended precision (128 bits)
a128 = np.array([1.], dtype=np.float128)
b128 = np.array([3000.001], dtype=np.float128)
c128 = np.array([3.], dtype=np.float128)
x1_128 = np.array([1.], dtype=np.float128)
x2_128 = np.array([1.], dtype=np.float128)

x1_128 = (-b128 + np.sqrt(b128**2 - 4*a128*c128))/(2*a128)
x2_128[0] = (-b128[0] - np.sqrt(b128[0]**2 - 4*a128[0]*c128[0]))/(2*a128[0])

print(x1_128[0])
print(x2_128) 

-0.0010000000000000008882
[-3000.]


# Erros de Truncamento e Séries de Taylor

Os erros de truncamento são aqueles que resultam do uso de uma aproximação no lugar
de um procedimento matemático exato. Por exemplo, aproximação dederivada pode ser feita por uma equação de diferenças divididas da forma 
$$
\frac{dx}{dt} \approx \frac{\Delta x}{\Delta t} = \frac{x(t_{i+1}) - x(t_i)}{t_{i+1} - t_i}
$$

Um erro de truncamento é introduzido na solução numérica porque a equação de diferenças apenas aproxima o valor verdadeiro da derivada. Para obter uma percepção das propriedades de tais erros, será abordada agora uma formulação matemática que é amplamente usada nos métodos numéricos para expressar uma função de forma aproximada — a série de Taylor. 

## A Série de Taylor
Se a função f e suas primeiras $n +1$ derivadas forem contínuas em um intervalo contendo $a$ e $x$, então o valor da função em $x$ é dado por

$$
f(x) = f(a) + f'(a)(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f^{(3)}(a)}{3!}(x-a)^3 + \cdot\cdot\cdot + \frac{f^{(n)}(a)}{n!}(x-a)^n + R_n
$$
onde o resto $R_n$ é definido por 
$$
R_n = \int^x_a \frac{(x-t)^n}{n!} f^{(n+1)}(t)dt = \frac{f^{(n+1)}(\xi)}{(n+1)!}(x-a)^{n+1}
$$

Em um calculo iterativo é conveniente simplificar a série de Taylor definindo um tamanho do
passo $h = x_{i+1} - x_i$ e expressando a equação geral como
$$
f(x_{i+1}) = f(x_{i}) + f'(x_i)h + \frac{f''(x_i)}{2!}h^2 + \frac{f^{(3)}(x_i)}{3!}h^3 + \cdot\cdot\cdot + \frac{f^{(n)}(x_i)}{n!}h^n + R_n
$$
onde o resto agora é 

$$
R_n =  \frac{f^{(n+1)}(\xi)}{(n+1)!}h^{n+1}
$$

Agora vamos escrever um script que calcule a série de Taylor para uma função. Vamos usar o Sympy para fazer isso. Basicamente a ideia é que a gente sabe o valor de uma função e as suas derivadas em um ponto ($x0$) e queremos saber o valor aproximado da em um ponto proximo.   

In [30]:
import sympy as sp
x, t = sp.symbols('x t')

def taylor_expansion(func, x0, order):
    """
    func: a sympy univariable function with x as its argument. 
    order: the order of expansion
    -----
    returns the expansion of the `func` around the point `a` at the order `order` 
    """
    series = 0
    for i in range(order+1):
        series += (func.diff(x, i).subs(x,x0)/sp.factorial(i))*(x-x0)**i
    
    return series


Vamos testar o nosso script para aproximar a função polinomial na vizinhaça do ponto $x_0=0$

$$
f(x) = -0.1 x^4 - 0.15 x^3 - 0.5 x^2 - 0.25 x + 1.2
$$

In [37]:
f = -0.1*x**4 - 0.15*x**3 - 0.5*x**2 - 0.25*x + 1.2

display(f)

-0.1*x**4 - 0.15*x**3 - 0.5*x**2 - 0.25*x + 1.2

A primeira aproxação é 

In [38]:
taylor_expansion(f, 0, 1)

1.2 - 0.25*x

Para achar o erro relativo verdadeiro dessa aproximação no ponto $x=1$ na primeira ordem temos

In [39]:
e_t_1 = (f.subs(x,1) - taylor_expansion(f, 0, 1).subs(x,1))/ f.subs(x,1)

display(e_t_1)

-3.75000000000000

E na segunda ordem

In [42]:
e_t_2 = (f.subs(x,1) - taylor_expansion(f, 0, 2).subs(x,1))/ f.subs(x,1)

display(e_t_2)

-1.25000000000000

Na terceira ordem

In [43]:
e_t_3 = (f.subs(x,1) - taylor_expansion(f, 0, 3).subs(x,1))/ f.subs(x,1)

display(e_t_3)

-0.500000000000000

Podemos plotar essas aproximações

In [52]:
xx = np.linspace(0,3,20)

y_real = [float(f.subs(x, i)) for i in xx]
y0 = [float(taylor_expansion(f, 0, 0).subs(x, i)) for i in xx]
y1 = [float(taylor_expansion(f, 0, 1).subs(x, i)) for i in xx]
y2 = [float(taylor_expansion(f, 0, 2).subs(x, i)) for i in xx]
y3 = [float(taylor_expansion(f, 0, 3).subs(x, i)) for i in xx]
y4 = [float(taylor_expansion(f, 0, 4).subs(x, i)) for i in xx]

In [53]:
fig = go.Figure()
fig.add_scatter(x = xx, y = y_real, name = 'real values')
fig.add_scatter(x = xx, y = y0, name = 'zero order')
fig.add_scatter(x = xx, y = y1, name = 'first order')
fig.add_scatter(x = xx, y = y2, name = 'second order')
fig.add_scatter(x = xx, y = y3, name = 'third order')
fig.add_scatter(x = xx, y = y4, name = 'forth order')


Como essa função é um polinomial de quarta grau a derivada de ordens maior do que 4 são zeros. Por isso, os termos sa série de Taylor que tem essas derivadas serão nulos e a série termina em 5 termos.

Agora vamos estudar uma função que precise de infinitos termos na sua série de Taylor.

Vamos ver o comportamento da $f(x) = \cos(x)$ no ponto $x = \frac{\pi}{3}$ baseado em valores da função e suas deerivadas no ponto $x = \frac{\pi}{4}$.

In [55]:
f = sp.cos(x)

xx = np.linspace(1,np.pi,20)

y_real = [float(f.subs(x, i)) for i in xx]
y0 = [float(taylor_expansion(f, np.pi/4, 0).subs(x, i)) for i in xx]
y1 = [float(taylor_expansion(f, np.pi/4, 1).subs(x, i)) for i in xx]
y2 = [float(taylor_expansion(f, np.pi/4, 2).subs(x, i)) for i in xx]
y3 = [float(taylor_expansion(f, np.pi/4, 3).subs(x, i)) for i in xx]
y4 = [float(taylor_expansion(f, np.pi/4, 4).subs(x, i)) for i in xx]
y10 = [float(taylor_expansion(f, np.pi/4, 10).subs(x, i)) for i in xx]


fig = go.Figure()
fig.add_scatter(x = xx, y = y_real, name = 'real values')
fig.add_scatter(x = xx, y = y0, name = 'zero order')
fig.add_scatter(x = xx, y = y1, name = 'first order')
fig.add_scatter(x = xx, y = y2, name = 'second order')
fig.add_scatter(x = xx, y = y3, name = 'third order')
fig.add_scatter(x = xx, y = y4, name = 'forth order')
fig.add_scatter(x = xx, y = y10, name = 'tenth order')

## Propagação de erros

Para $n$ variaveis independentes $x_1, x_2, ...,x_n$ tendo erros $\Delta \tilde{x}_1, \Delta \tilde{x}_2, ..., \Delta \tilde{x}_n$, o erro da função $f$ pode ser estimado por

$$
\Delta f = |\frac{\partial f}{\partial x_1}| \Delta \tilde{x}_1 + |\frac{\partial f}{\partial x_2}| \Delta \tilde{x}_2 + ... + |\frac{\partial f}{\partial x_n}| \Delta \tilde{x}_n
$$